![NYC Skyline](img/nyc.jpg)

Welcome to New York City, one of the most-visited cities in the world. There are many [Airbnb](https://www.airbnb.com/) listings in New York City to meet the high demand for temporary lodging for travelers, which can be anywhere between a few nights to many months. In this notebook, we will take a closer look at the New York Airbnb market by combining data from multiple file types like `.csv`, `.tsv`, and `.xlsx`.

Recall that **CSV**, **TSV**, and **Excel** files are three common formats for storing data. 
Three files containing data on 2019 Airbnb listings are available to you:

**data/airbnb_price.csv**
- **`listing_id`**: unique identifier of listing
- **`price`**: nightly listing price in USD
- **`nbhood_full`**: name of borough and neighborhood where listing is located

**data/airbnb_room_type.xlsx**
This is an Excel file containing data on Airbnb listing descriptions and room types.
- **`listing_id`**: unique identifier of listing
- **`description`**: listing description
- **`room_type`**: Airbnb has three types of rooms: shared rooms, private rooms, and entire homes/apartments

**data/airbnb_last_review.tsv**
This is a TSV file containing data on Airbnb host names and review dates.
- **`listing_id`**: unique identifier of listing
- **`host_name`**: name of listing host
- **`last_review`**: date when the listing was last reviewed

Our goals are to convert untidy data into appropriate formats to analyze, and answer key questions including:

- What is the average price, per night, of an Airbnb listing in NYC?
- How does the average price of an Airbnb listing, per month, compare to the private rental market?
- How many adverts are for private rooms?
- How do Airbnb listing prices compare across the five NYC boroughs?


In [9]:
# Dependencies
import numpy as np
import pandas as pd

In [10]:
# Load in datasets
prices = pd.read_csv('data/airbnb_price.csv')

xls = pd.ExcelFile('data/airbnb_room_type.xlsx')

room_types = xls.parse(0)

reviews = pd.read_csv('data/airbnb_last_review.tsv', sep='\t')

In [11]:
# Preview the prices df
prices.head()

,listing_id,price,nbhood_full
0,2595,225 dollars,"Manhattan, Midtown"
1,3831,89 dollars,"Brooklyn, Clinton Hill"
2,5099,200 dollars,"Manhattan, Murray Hill"
3,5178,79 dollars,"Manhattan, Hell's Kitchen"
4,5238,150 dollars,"Manhattan, Chinatown"


In [12]:
# Remove whitespace and string characters trailing the numeric values in price column
prices['price'] = prices['price'].str.replace(" dollars", "")

# Convert price column to numeric data type
prices['price'] = pd.to_numeric(prices['price'])

In [13]:
# Get a description of prices df
prices.describe()

,listing_id,price
count,2.520900e+04,25209.000000
mean,2.068922e+07,141.777936
std,1.102928e+07,147.349137
min,2.595000e+03,0.000000
25%,1.202273e+07,69.000000
50%,2.234391e+07,105.000000
75%,3.037669e+07,175.000000
max,3.645581e+07,7500.000000


In [27]:
# Remove outliers 
free_listing = prices['price'] == 0

prices = prices.loc[~free_listing]

In [28]:
# Calculate the average price
avg_price = round(prices['price'].mean(), 2)

# Display the results
avg_price

141.82

In [29]:
# Add a new column price_per_month to prices dataframe
prices['price_per_month'] = (prices['price'] * 365) / 12